Imports

In [1]:
import numpy as np
import tensorflow as tf

Load the data

In [2]:
# Load the training data first:
npz = np.load('financial_data_train.npz')

train_inputs = npz['inputs'].astype(np.float64)
train_targets = npz['targets'].astype(np.int32)

# Load the validation data next:
npz = np.load('financial_data_validation.npz')

validation_inputs = npz['inputs'].astype(np.float64)
validation_targets = npz['targets'].astype(np.int32)

# Finally, load the test data:
npz = np.load('financial_data_test.npz')

test_inputs = npz['inputs'].astype(np.float64)
test_targets = npz['targets'].astype(np.int32)

Neural Network model

In [7]:
input_size = 47 # Number of features in the input data set.
output_size = 2 # Since the outcome is binary.

hidden_layer_size = (input_size + output_size) // 2

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(input_size,)),  # Input layer
    tf.keras.layers.Dense(hidden_layer_size, activation= 'relu'), # Hidden Layer 1
    tf.keras.layers.Dropout(0.5), 
    tf.keras.layers.Dense(hidden_layer_size, activation= 'relu'), # Hidden Layer 2
    tf.keras.layers.Dropout(0.5), 
    tf.keras.layers.Dense(hidden_layer_size, activation= 'relu'), # Hidden Layer 3
    tf.keras.layers.Dropout(0.5), 
    tf.keras.layers.Dense(output_size, activation= 'softmax') # Output layer
])

optimizer = tf.keras.optimizers.SGD(learning_rate= 0.01, momentum= 0.9)

model.compile(
    optimizer= 'adam',
    loss= 'sparse_categorical_crossentropy', # Since the outcome is binary (0 or 1) 
    metrics= ['accuracy'] # parameters over which the model accuracy is evaluated
)

batch_size = 64
max_epochs = 100 # Training will stop after 100 cycles if callback doesn't kick in.

# Implementing callback -> EarlyStopping with a patience of 10 so that
# if the validation accuracy doesn't increase for 10 consecutive trainin
# epochs we can stop the training of model.
# This helps to prevent overfitting.
early_stopping = tf.keras.callbacks.EarlyStopping(monitor= 'val_accuracy', patience= 5, restore_best_weights= True)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor= 'val_accuracy', factor= 0.2, patience= 2, min_lr= 0.001)

model.fit(
    train_inputs,
    train_targets,
    batch_size = batch_size,
    epochs= max_epochs,
    callbacks= [early_stopping, reduce_lr],
    validation_data= (validation_inputs, validation_targets),
    verbose= 2
)

Epoch 1/100
1303/1303 - 3s - 2ms/step - accuracy: 0.5062 - loss: 0.6958 - val_accuracy: 0.5128 - val_loss: 0.6929 - learning_rate: 0.0010
Epoch 2/100
1303/1303 - 1s - 1ms/step - accuracy: 0.5164 - loss: 0.6925 - val_accuracy: 0.5171 - val_loss: 0.6921 - learning_rate: 0.0010
Epoch 3/100
1303/1303 - 1s - 1ms/step - accuracy: 0.5337 - loss: 0.6890 - val_accuracy: 0.5333 - val_loss: 0.6885 - learning_rate: 0.0010
Epoch 4/100
1303/1303 - 1s - 1ms/step - accuracy: 0.5437 - loss: 0.6840 - val_accuracy: 0.5441 - val_loss: 0.6814 - learning_rate: 0.0010
Epoch 5/100
1303/1303 - 1s - 1ms/step - accuracy: 0.5494 - loss: 0.6799 - val_accuracy: 0.5452 - val_loss: 0.6787 - learning_rate: 0.0010
Epoch 6/100
1303/1303 - 1s - 1ms/step - accuracy: 0.5563 - loss: 0.6759 - val_accuracy: 0.5485 - val_loss: 0.6762 - learning_rate: 0.0010
Epoch 7/100
1303/1303 - 1s - 1ms/step - accuracy: 0.5606 - loss: 0.6735 - val_accuracy: 0.5457 - val_loss: 0.6744 - learning_rate: 0.0010
Epoch 8/100
1303/1303 - 1s - 1ms/s